In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
! pip install transformers -q

     |████████████████████████████████| 389kB 9.7MB/s 
     |████████████████████████████████| 1.0MB 38.3MB/s 
     |████████████████████████████████| 675kB 64.6MB/s 
     |████████████████████████████████| 860kB 56.6MB/s 


In [0]:
import csv
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.metrics import *
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [0]:
import os
os.chdir('/gdrive/My Drive/Definition')

In [0]:
model_name="bert-base-uncased"
#model_name="roberta-base"
#model_name="xlnet-base-cased"

In [8]:
merged = pd.read_csv('WCL_data2.csv')
merged.head(2)

,label,Psentence,Length,Dep1,Dep2,DepLabel
0,1,a pianist is a person who plays the piano.,9,pianist is is is person person plays plays piano,a pianist person . a plays who piano the,nsubj ROOT ROOT ROOT attr attr relcl relcl dobj
1,0,pingala has a sunlike nature and male energy.,8,has has has nature nature nature nature energy,pingala nature . a sunlike and energy male,ROOT ROOT ROOT dobj dobj dobj dobj conj


In [0]:
def seq_len(row):
  return len(row['Psentence'].split())

merged['Length'] = merged.apply(seq_len, axis=1)

In [10]:
merged = merged[merged['Length']>4]
merged.shape

(4680, 6)

In [0]:
label_cols = list(set(merged['label']))

## Transformer

In [12]:
 %tensorflow_version 1.x 
from transformers import BertTokenizer, GPT2Tokenizer, AutoTokenizer
from transformers import BertConfig, BertForSequenceClassification, AutoModel, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name, output_attentions=True)

## Preprocessing

In [13]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64
X = [tokenizer.encode(x, add_special_tokens=True) for x in merged['Psentence']]
X = pad_sequences(X, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Using TensorFlow backend.


In [0]:
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in X:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
attention_masks = np.array(attention_masks)

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [0]:
y = np.array(merged['label'])

# Training

In [17]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [0]:
from sklearn.utils.extmath import softmax
from sklearn.metrics import classification_report, f1_score


def flat_accuracy(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)


In [19]:
from sklearn.model_selection import KFold,StratifiedKFold
from tqdm import tqdm

auto_model = None

kf = StratifiedKFold(10, shuffle=True, random_state=2018)
for f,(tr, val) in enumerate(kf.split(X, y)):
    print(f'--------------------------------------------------------------------------------Fold # {f}--------------------------------------------------------------------')

    X_train, X_val = X[tr], X[val]
    y_train, y_val = y[tr], y[val]
    X_train_masks, X_val_masks = attention_masks[tr], attention_masks[val]
                                         
    # Convert all of our data into torch tensors, the required datatype for our model
    X_train = torch.tensor(X_train)
    X_val = torch.tensor(X_val)

    y_train = torch.tensor(y_train)
    y_val = torch.tensor(y_val)

    X_train_masks = torch.tensor(X_train_masks)
    X_val_masks = torch.tensor(X_val_masks)

    batch_size = 32

    # Create an iterator of our data with torch DataLoader 
    train_data = TensorDataset(X_train, X_train_masks, y_train)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(X_val, X_val_masks, y_val)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


    ## Model
    del auto_model
    auto_model = AutoModelForSequenceClassification.from_pretrained(model_name, output_attentions=True, num_labels= len(np.unique(y_train)))
    auto_model.to(device)

    # Optimizer
    num_total_steps = 1000
    num_warmup_steps = 100
    lr = 2e-5
    param_optimizer = list(auto_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    

    auto_model.train()  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    n_epochs = 2

    for epoch in (range(n_epochs)):  
      # Training Loop
      for step, batch in (enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()
        outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
          
      auto_model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # Validation Loop
      yt, yp = [], []
      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          outputs = auto_model(b_input_ids, 
                              token_type_ids=None,
                              attention_mask=b_input_mask,
                              labels= b_labels)        
          loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        
        preds = softmax(logits)
        pred_ids = np.argmax(preds, axis=1).flatten()

        label_ids = b_labels.to('cpu').numpy()

        eval_loss += loss.item()
        eval_accuracy += flat_accuracy(logits, label_ids)

        yt = yt + label_ids.tolist()
        yp = yp + pred_ids.tolist()

        nb_eval_steps +=1
        
      print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                      tr_loss/nb_tr_steps,
                                                                      eval_loss/nb_eval_steps, 
                                                                    ))
    #if epoch == n_epochs:
    print(classification_report(yt, yp))   


--------------------------------------------------------------------------------Fold # 0--------------------------------------------------------------------



Epoch 0 | Train loss: 0.1585796523952123 | Validation Loss: 0.09802586423854033 
Epoch 1 | Train loss: 0.10284176971728096 | Validation Loss: 0.07202956229448318 
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       281
           1       0.97      0.97      0.97       188

    accuracy                           0.98       469
   macro avg       0.98      0.98      0.98       469
weighted avg       0.98      0.98      0.98       469

--------------------------------------------------------------------------------Fold # 1--------------------------------------------------------------------
Epoch 0 | Train loss: 0.17176300339457212 | Validation Loss: 0.07631193213164807 
Epoch 1 | Train loss: 0.11623833613173867 | Validation Loss: 0.03781630769371987 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       281
           1       0.98      1.00      0.99       188

    accuracy            

In [0]:
torch.save(auto_model.state_dict(), 'models/bert_attn_97_10fold.pth')

In [0]:
# auto_model.load_state_dict(torch.load('models/bert_new.pth'))
# print(f'Done')

# Analysis of model

In [0]:
y_pred = []
y_true = []

x = []

for batch in tqdm(validation_dataloader):
  
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
    logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  prob = softmax(logits)
  pred = np.argmax(prob, axis=1)
  for i in b_input_ids:
    z = []
    for j in i:
      z.append(j.item())
    x.append(z)
  [y_pred.append(i) for i in prob[:,1]]
  [y_true.append(i) for i in label_ids]

In [0]:
df = pd.DataFrame({'Prob':y_pred, 'class':y_true, 'input':x})
df.head()

In [0]:
#tok1.decode(df.loc[1,'input'])

df['sentence'] = df['input'].apply(lambda x:tok1.decode(x))
df.head()

In [0]:
df['Correct'] = ((df['Prob']>=0.5)*1 == df['class'])*1
w = df[df['Correct']==0].copy()
w.head()

In [0]:
w.sort_values('Prob', inplace=True)
w.shape

In [0]:
fp = w[w['class']==0].reset_index(drop=True)
fn = w[w['class']==1].reset_index(drop=True)

In [0]:
fp.head()

In [0]:
for i in range(10):
  print(fp.loc[i,'sentence'])

In [0]:
for i in range(10):
  print(fn.loc[i,'sentence'])